# The Analysis of several Binary Star Configurations to find the B-Spline Signal

In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import numpy as np # main library for numeric calculations
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting
import seaborn as sns # Another library for data plotting with more functions
sns.set()

import scipy as scp # STEM software
from scipy import signal
import scipy.interpolate as interpolate

from glob import glob # check files
from IPython.display import display, Markdown

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import lightkurve as lk
import astropy

import sys, os

file_path = os.getcwd()
print(file_path)
parent_dir = "\\".join(file_path.split("\\")[:-1])
print(parent_dir)

sys.path.append(parent_dir)

from kepler import *

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

import warnings
warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter


def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))


c:\users\prannaya\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


C:\Users\Prannaya\Documents\GitHub\ThreeBody\notebooks
C:\Users\Prannaya\Documents\GitHub\ThreeBody


In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
!pip install ipympl
%matplotlib widget

# Testing matplotlib interactions with a simple plot
fig = plt.figure()
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.toolbar_visible = True

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
binarystars = pd.read_csv("../data/binarystars.csv").rename(columns={"KIC": "id"}).set_index("id")
configs = pd.read_csv("../data/binaryconfigs.csv")
configs.columns = ["id", "period", "duration", "depth"]
configs = configs.set_index("id")

### Kepler Eclipsing Binary Catalog

In [5]:
binarystars

,period,period_err,bjd0,bjd0_err,morph,GLon,GLat,kmag,Teff,SC
id,,,,,,,,,,
3863594,0.053268,0.0,55000.000000,0.004327,0.79,-1.0000,-1.0000,-1.000,-1.0,False
10417986,0.073731,0.0,55000.027476,0.004231,0.99,81.0390,11.0820,9.128,-1.0,True
8912468,0.094838,0.0,54953.576945,0.005326,0.98,80.1095,7.8882,11.751,6194.0,False
8758716,0.107205,0.0,54953.672989,0.006197,1.00,77.7478,11.6565,13.531,-1.0,False
10855535,0.112782,0.0,54964.629315,0.006374,0.99,79.3949,15.9212,13.870,7555.0,False
...,...,...,...,...,...,...,...,...,...,...
9408440,989.985000,-1.0,55346.365980,0.096130,0.00,78.5607,12.2615,13.199,5688.0,False
8054233,1058.000000,-1.0,54751.806288,0.968052,0.03,78.6142,7.7321,11.783,4733.0,False
7672940,1064.270000,-1.0,54977.092960,0.089646,0.00,74.5296,14.6136,12.328,-1.0,False


In [6]:
def day2seconds(s):
    if type(s) in [float, int]:
        return s * 86400
    return float(s[:-2]) * 86400

configs["period"] = configs.period.apply(day2seconds)
configs["duration"] = configs.duration.apply(day2seconds)
configs = configs.dropna().sort_index()
configs

,period,duration,depth
id,,,
1295531,72815.186670,28512.0,0.007980
1572353,39557.400754,4320.0,0.047896
1573836,102536.652246,28512.0,0.000168
1868650,28628.333735,28512.0,0.000121
2012362,33343.846896,28512.0,0.011891
...,...,...,...
12508348,33115.084997,28512.0,0.002751
12554536,59151.892424,8640.0,0.227355
12598713,33328.499216,28512.0,0.011091


## EDA
Exploratory Data Analysis

### Constants

In [7]:
n = len(configs)
bins = int(np.sqrt(n))

periods = configs.period
durations = configs.duration
depths = configs.depth

percentiles = np.array([2.5, 25, 50, 75, 97.5])

### Summary Statistics

In [8]:
mu_period = periods.mean()
sigma_period = periods.std()
var_period = np.var(periods)
ptiles_per = np.percentile(periods, percentiles)

mu_duration = durations.mean()
sigma_duration = durations.std()
var_duration = np.var(durations)
ptiles_dur = np.percentile(durations, percentiles)

mu_depth = depths.mean()
sigma_duration = depths.std()
var_depth = np.var(depths)
ptiles_dep = np.percentile(depths, percentiles)

### Functions

In [9]:
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

def plot_hist(data, xlabel, ylabel="count", bins=bins, ax=None, **kwargs):
    if not ax:
        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111)
    
    ax.hist(data, bins=bins, **kwargs)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()

    
def plot_ecdf(data, xlabel, ptiles, ylabel="ECDF", ax=None, marker=".", linestyle="none", **kwargs):
    if not ax:
        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111)
        
    # Retrive ECDF
    x, y = ecdf(data)

    # Generate plot
    ax.plot(x, y, marker=marker, linestyle=linestyle, **kwargs)

    # Label the axes
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    ax.plot(ptiles, percentiles/100, marker='D', color='red',
         linestyle="none")

    # Display the plot
    plt.show()

### Graphical EDA
#### Histograms

In [10]:
plot_hist(periods, "Period")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plot_hist(durations, "Duration")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plot_hist(depths, "Depth")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### ECDF

In [13]:
plot_ecdf(periods, "Period", ptiles_per)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_ecdf(durations, "Duration", ptiles_dur)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_ecdf(depths, "Depth", ptiles_dep)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Quantitative EDA

#### Covariance

In [16]:
cov_perdur = np.cov(periods, durations)
cov_perdep = np.cov(periods, depths)
cov_durdep = np.cov(durations, depths)

#### Correlation

In [17]:
configs.corr()

,period,duration,depth
period,1.000000,-0.202105,0.019205
duration,-0.202105,1.000000,-0.180927
depth,0.019205,-0.180927,1.000000


In [18]:
plt.figure(figsize=(16, 12))
heatmap = sns.heatmap(configs.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Pearson Coefficient

In [19]:
def pearson_r(x, y):
    """Compute Pearson correlation coefficient between two arrays."""
    corr_mat = np.corrcoef(x, y)
    return corr_mat[0,1]

pearson_perdur = pearson_r(periods, durations)
pearson_perdep = pearson_r(periods, depths)
pearson_durdep = pearson_r(durations, depths)

## Signal Processing

In [29]:
def evaluateAndProcessData(x, y):
    t, c, k = interpolate.splrep(x, y, s=0, k=4)
    xx = np.linspace(x.min(), x.max(), 100)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    
    plt.figure(figsize=(16,6))
    plt.plot(x, y, 'bo', label='Original points')
    plt.plot(x, y)
    plt.plot(xx, spline(xx), 'r', label='BSpline')
    plt.grid()
    plt.legend(loc='best')
    plt.show()
    
# Retrieving all the important details
def getData(id):
    try:
        return retrieveKeplerLightCurve(id)
    except:
        return None


In [31]:
ids = list(binaryconfigs.index)
id2 = np.random.choice(ids)
id2

7449844

In [32]:
lc = getData(id2)
lc

time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,sap_quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,,,,d,pix,pix,,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
object,float32,float32,int32,float32,float64,float64,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
131.51226382373716,1.0061246e+00,1.2709832e-04,0,1.270004e-03,360.91046,511.40898,1105,5.0660051e+04,6.4205713e+00,1.1918115e+03,3.4365416e-01,5.3879867e+04,6.8063540e+00,0,nan,nan,nan,nan,360.91046,1.4274896e-04,511.40898,1.4618262e-04,-2.9109626e-03,-1.4209282e-02
131.53269840010762,1.0067688e+00,1.2709244e-04,0,1.270880e-03,360.91083,511.40919,1106,5.0671379e+04,6.4210439e+00,1.1904001e+03,3.4367025e-01,5.3914363e+04,6.8060393e+00,0,nan,nan,nan,nan,360.91083,1.4284824e-04,511.40919,1.4614535e-04,-2.9087204e-03,-1.4233068e-02
131.5531328764846,1.0066149e+00,1.2709836e-04,0,1.271756e-03,360.91068,511.40936,1107,5.0665672e+04,6.4208636e+00,1.1920410e+03,3.4352806e-01,5.3906125e+04,6.8063564e+00,0,nan,nan,nan,nan,360.91068,1.4272633e-04,511.40936,1.4619122e-04,-2.7791162e-03,-1.4089028e-02
131.5735672526207,1.0070796e+00,1.2711562e-04,0,1.272633e-03,360.91074,511.40945,1108,5.0693184e+04,6.4221277e+00,1.1912972e+03,3.4384197e-01,5.3931008e+04,6.8072805e+00,0,nan,nan,nan,nan,360.91074,1.4264000e-04,511.40945,1.4608621e-04,-2.5506532e-03,-1.3908224e-02
131.59400172888127,1.0077182e+00,1.2727438e-04,0,1.273509e-03,360.91031,511.40829,1109,5.0727152e+04,6.4237919e+00,1.1916511e+03,3.4402812e-01,5.3965207e+04,6.8157830e+00,0,nan,nan,nan,nan,360.91031,1.4257627e-04,511.40829,1.4599851e-04,-2.7189055e-03,-1.4636266e-02
131.61443630502617,1.0082084e+00,1.2725357e-04,0,1.274385e-03,360.91091,511.40860,1110,5.0756348e+04,6.4252353e+00,1.1907670e+03,3.4384146e-01,5.3991457e+04,6.8146682e+00,0,nan,nan,nan,nan,360.91091,1.4246817e-04,511.40860,1.4591723e-04,-2.7469283e-03,-1.4556367e-02
131.63487068105314,1.0081728e+00,1.2726043e-04,0,1.275261e-03,360.91062,511.40887,1111,5.0759992e+04,6.4253817e+00,1.1892471e+03,3.4389180e-01,5.3989551e+04,6.8150358e+00,0,nan,nan,nan,nan,360.91062,1.4245333e-04,511.40887,1.4590398e-04,-2.7854952e-03,-1.4465231e-02
131.6553051570736,1.0084233e+00,1.2736068e-04,0,1.276137e-03,360.91022,511.40841,1112,5.0777312e+04,6.4261627e+00,1.1884214e+03,3.4382558e-01,5.4002965e+04,6.8204041e+00,0,nan,nan,nan,nan,360.91022,1.4243464e-04,511.40841,1.4584478e-04,-2.9709986e-03,-1.4714237e-02


In [33]:
plotKeplerLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plotKeplerSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plotKeplerPDCSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
time = np.array(list(map(lambda time: time.value, list(lc.time))))
time

array([ 131.51226382,  131.5326984 ,  131.55313288, ..., 1590.96040616,
       1590.98084053, 1591.00127509])

In [41]:
flux = np.array(list(map(lambda flux: flux.value, list(lc.flux))))
flux

array([1.0061246 , 1.0067688 , 1.0066149 , ..., 0.99598616, 0.9954713 ,
       0.9959612 ], dtype=float32)

In [42]:
evaluateAndProcessData(time, flux)

ValueError: Error on input data